In [32]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [33]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.00045893s, CPU 0.000456s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0124735s, CPU 0.011037s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [34]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:AxiSymmetric)

Problem("body1-2D", :AxiSymmetric, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 98)

In [35]:
supp = FEM.displacementConstraint("left", ux=0, uy=0)
load = FEM.load("right", fx=1)

("right", 1, 0, 0)

In [36]:
u0 = FEM.initialDisplacement(problem, "body", ux=0, uy=0)

VectorField

196×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [37]:
FEM.initialDisplacement!(problem, "left", u0, ux=0, uy=0)
u0

MethodError: MethodError: no method matching initialDisplacement!(::Problem, ::String, ::VectorField; ux::Int64, uy::Int64)
The function `initialDisplacement!` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  initialDisplacement!(::Any, ::Any; ux, uy, uz)
   @ LowLevelFEM ~/Dokumentumok/GitHub/LowLevelFEM.jl/src/linear.jl:5270


In [38]:
v0 = FEM.initialVelocity(problem, "body", vx=0, vy=0)

VectorField

196×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [39]:
FEM.initialVelocity!("body", v0, vx=0, vy=0)

In [40]:
K = FEM.stiffnessMatrix(problem)
M = FEM.massMatrix(problem)
f = FEM.loadVector(problem, [load])

VectorField

196×1 Matrix{Float64}:
 0.0
 0.0
 0.39269908169872364
 0.0
 0.39269908169872586
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [41]:
FEM.applyBoundaryConditions!(K, M, f, [supp])

In [42]:
Tmin = FEM.smallestPeriodTime(K, M)

4.282263524466636e-8

In [43]:
u, v = FEM.CDM(K, M, f, u0, v0, 10Tmin, Tmin / π)

VectorField

196×32 Matrix{Float64}:
 0.0  0.0          0.0          0.0         …   0.0          0.0
 0.0  0.0          0.0          0.0             0.0          0.0
 0.0  2.01762e-7   6.06278e-7   9.77961e-7      3.08195e-6   2.97149e-6
 0.0  0.0          1.24238e-7   3.601e-7        8.37878e-7   5.48817e-7
 0.0  2.01762e-7   6.0749e-7    9.74008e-7      3.13395e-6   2.99303e-6
 0.0  0.0         -1.20354e-7  -3.57607e-7  …  -8.18659e-7  -5.58659e-7
 0.0  0.0          0.0          0.0             0.0          0.0
 0.0  0.0          0.0          0.0             0.0          0.0
 0.0  0.0          0.0          0.0             8.3774e-7    7.18592e-7
 0.0  0.0          0.0          0.0             2.36919e-6   2.28505e-6
 ⋮                                          ⋱   ⋮           
 0.0  0.0          0.0          0.0            -1.90052e-6  -1.75778e-6
 0.0  0.0          0.0          0.0             2.83936e-6   2.60713e-6
 0.0  0.0          0.0          0.0            -1.94444e-6  -1.77408e-6
 0.0  0

VectorField

196×32 Matrix{Float64}:
 0.0   0.0      0.0        0.0       …    0.0        0.0        0.0
 0.0   0.0      0.0        0.0            0.0        0.0        0.0
 0.0  14.8018  29.6765    27.2677       -36.1018   -23.2071    -8.10409
 0.0   0.0      9.11447   17.3035       -25.759    -19.9353   -21.2064
 0.0  14.8018  29.7654    26.8889       -32.9414   -23.2074   -10.338
 0.0   0.0     -8.8295   -17.4056    …   24.1745    20.1215    19.0743
 0.0   0.0      0.0        0.0            0.0        0.0        0.0
 0.0   0.0      0.0        0.0            0.0        0.0        0.0
 0.0   0.0      0.0        0.0           -6.95185  -13.211     -8.74105
 0.0   0.0      0.0        0.0           -2.23073   -4.22233   -6.17326
 ⋮                                   ⋱               ⋮        
 0.0   0.0      0.0        0.0           10.5682    10.9096    10.4716
 0.0   0.0      0.0        0.0           -3.89859  -10.3448   -17.0369
 0.0   0.0      0.0        0.0           13.8807     8.20835   12.4984


(, )

In [44]:
FEM.showDoFResults(u, :uvec, visible=true)

0

In [45]:
FEM.showDoFResults(v, :vvec)

1

In [46]:
e1 = FEM.solveStrain(u)

TensorField

162-element Vector{Matrix{Float64}}:
 [0.0 0.0 … 3.5760857234376182e-6 3.5776392288356156e-6; 0.0 0.0 … 1.381719319916727e-6 6.883729177633206e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 5.263940016607379e-6 4.790102602245721e-6]
 [0.0 0.0 … 4.831474527863384e-6 4.2749515809470996e-6; 0.0 0.0 … 6.046148669035398e-7 7.477052500653154e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 4.906130075538841e-6 4.402881159260841e-6]
 [0.0 0.0 … 4.536500426209625e-6 5.140710339512859e-6; 0.0 0.0 … -6.750544137102992e-7 -6.987529555873798e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 5.5683927303032094e-6 5.005411762320361e-6]
 [0.0 0.0 … 6.932040936330064e-6 6.902503887604091e-6; 0.0 0.0 … -3.602120785082574e-7 -4.6196337892994256e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 6.014771259310582e-6 5.191691538754512e-6]
 [0.0 0.0 … 5.090681876932617e-6 5.751616912847466e-6; 0.0 0.0 … -7.306656924871694e-7 -7.036379353206922e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 5.710490628360037e-6 5.349746136948757e-6]
 [0.0 0.0 … 5.0404706024672845e-

In [47]:
FEM.showStrainResults(e1, :e)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


2

In [48]:
e2 = FEM.solveStrain(u, DoFResults=true)

TensorField

882×32 Matrix{Float64}:
 0.0  0.0          0.0           0.0         …   5.56243e-6   5.50099e-6
 0.0  0.0          0.0           0.0             1.69354e-5   1.74234e-5
 0.0  0.0          0.0           0.0             0.0          0.0
 0.0  0.0          0.0           0.0             1.69354e-5   1.74234e-5
 0.0  0.0          0.0           0.0            -8.78639e-7  -6.09222e-7
 0.0  0.0          0.0           0.0         …   0.0          0.0
 0.0  0.0          0.0           0.0             0.0          0.0
 0.0  0.0          0.0           0.0             0.0          0.0
 0.0  0.0          0.0           0.0             0.0          0.0
 0.0  2.19005e-6   4.50733e-6    4.13799e-6      1.46591e-6   2.82284e-6
 ⋮                                           ⋱   ⋮           
 0.0  0.0          2.64362e-7    1.37523e-6      2.69045e-6   2.60033e-6
 0.0  0.0         -4.1165e-9    -2.29259e-8     -5.87628e-7  -5.84259e-7
 0.0  0.0          0.0           0.0         …   0.0          0.0
 0.0  0

In [49]:
FEM.showDoFResults(e2, :e)

3

In [50]:
e3 = FEM.elementsToNodes(e1)

TensorField

882×32 Matrix{Float64}:
 0.0  0.0          0.0           0.0         …   5.56243e-6   5.50099e-6
 0.0  0.0          0.0           0.0             8.46771e-6   8.71171e-6
 0.0  0.0          0.0           0.0             0.0          0.0
 0.0  0.0          0.0           0.0             8.46771e-6   8.71171e-6
 0.0  0.0          0.0           0.0            -8.78639e-7  -6.09222e-7
 0.0  0.0          0.0           0.0         …   0.0          0.0
 0.0  0.0          0.0           0.0             0.0          0.0
 0.0  0.0          0.0           0.0             0.0          0.0
 0.0  0.0          0.0           0.0             0.0          0.0
 0.0  2.19005e-6   4.50733e-6    4.13799e-6      1.46591e-6   2.82284e-6
 ⋮                                           ⋱   ⋮           
 0.0  0.0          2.64362e-7    1.37523e-6      2.69045e-6   2.60033e-6
 0.0  0.0         -2.05825e-9   -1.14629e-8     -2.93814e-7  -2.92129e-7
 0.0  0.0          0.0           0.0         …   0.0          0.0
 0.0  0

In [51]:
FEM.showDoFResults(e3, :e)

4

In [52]:
s1 = FEM.solveStress(u)

TensorField

162-element Vector{Matrix{Float64}}:
 [0.0 0.0 … 1.0919246202611257 1.1538754426017006; 0.0 0.0 … 0.21257220306411184 0.10590352580974163; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.3928802739489974 1.3850876665476646]
 [0.0 0.0 … 1.4083021619902287 1.1787827110438611; 0.0 0.0 … 0.09301767183131382 0.11503157693312545; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.3785018684323909 1.1537847873631046]
 [0.0 0.0 … 1.516596437758631 1.6203085259009444; 0.0 0.0 … -0.10385452518619988 -0.10750045470575073; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.6589531886126254 1.5597624707988222]
 [0.0 0.0 … 2.0290695433130037 1.799188436233012; 0.0 0.0 … -0.05541724284742421 -0.071071289066145; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.9613100729768524 1.7250848481822585]
 [0.0 0.0 … 1.4484825572125741 1.5453751029767238; 0.0 0.0 … -0.1124101065364876 -0.10825199004933725; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.5602336610541192 1.5232797191417373]
 [0.0 0.0 … 1.7256740573444946 1.46317547228263; 0.0 0.0 … 0.10940907310500737 0.12523274432

In [53]:
FEM.showStressResults(s1, :s)

5

In [54]:
s2 = FEM.solveStress(u, DoFResults=true)

TensorField

882×32 Matrix{Float64}:
 0.0  0.0        0.0          …   1.61284     1.39619     1.41074
 0.0  0.0        0.0              1.27729     1.30272     1.34026
 0.0  0.0        0.0              0.0         0.0         0.0
 0.0  0.0        0.0              1.27729     1.30272     1.34026
 0.0  0.0        0.0              0.438421    0.405262    0.470709
 0.0  0.0        0.0          …   0.0         0.0         0.0
 0.0  0.0        0.0              0.0         0.0         0.0
 0.0  0.0        0.0              0.0         0.0         0.0
 0.0  0.0        0.0              0.615379    0.540437    0.564435
 0.0  0.612908   1.1989           0.643038    0.523899    0.935508
 ⋮                            ⋱               ⋮          
 0.0  0.0        0.0710938        0.913498    0.854656    0.893796
 0.0  0.0       -0.000316654     -0.0942556  -0.0452021  -0.044943
 0.0  0.0        0.0          …   0.0         0.0         0.0
 0.0  0.0       -0.000316654     -0.0942556  -0.0452021  -0.044943
 0.0  0.

In [55]:
FEM.showDoFResults(s2, :s)

6

In [56]:
err1 = FEM.fieldError(e1)

TensorField

882×32 Matrix{Float64}:
 0.0  0.0         0.0         …  1.65902e-6  1.13949e-6  2.4774e-7
 0.0  0.0         0.0            2.54396e-7  1.30427e-7  1.80741e-7
 0.0  0.0         0.0            0.0         0.0         0.0
 0.0  0.0         0.0            2.54396e-7  1.30427e-7  1.80741e-7
 0.0  0.0         0.0            1.15023e-6  8.78639e-7  6.09222e-7
 0.0  0.0         0.0         …  0.0         0.0         0.0
 0.0  0.0         0.0            0.0         0.0         0.0
 0.0  0.0         0.0            0.0         0.0         0.0
 0.0  0.0         0.0            0.0         0.0         0.0
 0.0  5.75954e-7  2.92263e-7     1.25963e-6  4.67808e-7  1.98176e-8
 ⋮                            ⋱              ⋮           
 0.0  0.0         3.74825e-7     3.58785e-7  8.45807e-7  9.08439e-7
 0.0  0.0         1.43028e-7     2.81032e-7  3.95437e-7  2.62779e-7
 0.0  0.0         0.0         …  0.0         0.0         0.0
 0.0  0.0         1.43028e-7     2.81032e-7  3.95437e-7  2.62779e-7
 0.0  0.0

In [57]:
FEM.showDoFResults(err1, :tensor)

7

In [58]:
u2, v2 = FEM.HHT(K, M, f, u0, v0, 10Tmin, Tmin / π)

VectorField

196×32 Matrix{Float64}:
 0.0   0.0           0.0           0.0          …   0.0          0.0
 0.0   0.0           0.0           0.0              0.0          0.0
 0.0   1.66718e-7    5.66996e-7    1.00939e-6       3.21666e-6   2.82394e-6
 0.0   1.70835e-8    1.04397e-7    2.96255e-7       9.52397e-7   7.03375e-7
 0.0   1.66716e-7    5.66556e-7    1.00655e-6       3.16339e-6   2.8372e-6
 0.0  -1.67178e-8   -1.02552e-7   -2.92484e-7   …  -9.71317e-7  -6.51553e-7
 0.0   0.0           0.0           0.0              0.0          0.0
 0.0   0.0           0.0           0.0              0.0          0.0
 0.0   1.08732e-14   2.96567e-13   3.96307e-12      1.08458e-6   1.06781e-6
 0.0   4.58426e-15   1.30617e-13   1.82464e-12      2.20452e-6   2.07657e-6
 ⋮                                              ⋱   ⋮           
 0.0  -3.55475e-14  -8.74652e-13  -1.04034e-11     -1.87704e-6  -1.6847e-6
 0.0   9.85216e-12   1.69497e-10   1.41748e-9       2.73945e-6   2.52782e-6
 0.0  -4.39858e-13  -8.37953e

VectorField

196×32 Matrix{Float64}:
 0.0   0.0           0.0          …    0.0        0.0        0.0
 0.0   0.0           0.0               0.0        0.0        0.0
 0.0  24.4618       34.2694          -30.3815   -31.2284   -26.394
 0.0   2.50659      10.3046          -21.3853   -20.0694   -16.4686
 0.0  24.4615       34.2052          -34.4288   -27.1441   -20.7159
 0.0  -2.45293     -10.1412       …   23.35      25.8289    21.0887
 0.0   0.0           0.0               0.0        0.0        0.0
 0.0   0.0           0.0               0.0        0.0        0.0
 0.0   1.59538e-6    4.03232e-5       12.4907     5.44836   -7.90899
 0.0   6.72629e-7    1.78196e-5      -16.38     -12.1857    -6.58682
 ⋮                                ⋱               ⋮        
 0.0  -5.21574e-6   -0.000117903      14.7992    14.1894    14.0323
 0.0   0.00144557    0.0219785       -11.2618   -16.1677   -14.8844
 0.0  -6.45385e-5   -0.00110042       15.6092    14.7981    12.3128
 0.0   0.0171543     0.198765     …  -16.86

(, )

In [59]:
FEM.showDoFResults(u2, :uvec)

8

In [60]:
FEM.showDoFResults(v2, :vvec)

9

In [61]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [62]:
gmsh.finalize()